# [실습] LangChain을 이용한 어플리케이션 개발   

LCEL 구조를 이용하여, 간단한 텍스트를 요약하는 어플리케이션을 만들어 보겠습니다.

In [ ]:
!pip install google_genai langchain langchain-community langchain_google_genai pymupdf

## Gemini API 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 https://aistudio.google.com  에 접속하면, API 키 생성이 가능합니다.

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter,
    temperature = 0.5,
    max_tokens = 2048
)

PDF 파일 준비하기   
임의의 PDF 파일을 다운로드하여 준비합니다.   
(예시 PDF 파일을 사용하실 분은 첨부된 PDF 파일을 활용해 주세요!)

In [ ]:
path_material = 'example.pdf'
# 자유롭게 경로 변경해서 실행하셔도 됩니다!

랭체인에서는 데이터를 `Document` 클래스로 처리합니다.   
데이터의 형식에 따라 적절한 document_loader를 불러와서 사용할 수 있습니다.   

이번 실습에서는 PDF를 불러오는 가장 간단한 로더인 PyMuPDFLoader를 사용합니다.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(path_material)
# 페이지별로 저장
pages = loader.load()
print("# Number of Pages:", len(pages))

파일이 너무 긴 경우,일부만 선택하여 요약할 수 있습니다.

In [ ]:
pages = pages[:11]

In [ ]:
pages[10]


PDF 파일은 페이지별 Document를 저장합니다.   
요약을 수행하기 위해, 전체 텍스트를 하나의 Document에 합칩니다.

In [ ]:
from langchain_core.documents import Document
# Document 클래스 만들기

corpus = Document(page_content='')
for page in pages:
    corpus.page_content += page.page_content + '\n'

corpus.page_content = corpus.page_content.replace('\n\n','\n')
len(corpus.page_content)

LLM에 처리하기 전, 토큰 수를 체크합니다.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client()


response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents=corpus.page_content
)

print("Prompt tokens:",response.total_tokens)

요약 체인을 만들고 구성합니다.

## [실습] 요약 체인 만들기

`corpus.page_content`를 입력으로 받는 요약 체인을 만들고 실행하세요.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
summary_prompt = ChatPromptTemplate(
    [('user', '''
당신은 LLM/AI 전문가입니다.
LLM에 대한 최신 모델의 논문이 주어집니다.
전문 개발자를 대상으로 이를 설명한다고 가정하고,
모델의 구조에 집중해서 논문의 내용을 요약하세요.

전체는 5문단으로 하고, 문단별 4~8개 문장으로 구성하세요.
---
{text}''')

    ]

)
summary_chain = summary_prompt | llm | StrOutputParser()

summary = summary_chain.invoke(corpus.page_content)
summary

Gemini 2.0 Flash의 Context Window는 1M이므로 전체를 모두 하나의 컨텍스트로 입력해도 되지만,   
Context가 짧은 모델들의 경우, 전체를 분할하여 요약 작업을 수행할 수 있습니다.

**Map-Reduce** 방식의 요약을 만들어 보겠습니다.   

Map-Reduce는 텍스트를 청크로 분할하고, 청크별 요약을 생성한 뒤  
전체 요약문을 합쳐 프롬프트로 넣고 최종 요약문을 생성하는 방식입니다.

문서를 청크로 나누기 위해, RecursiveCharacterTextSplitter를 사용합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,
    chunk_overlap=4000,
)

chunks = text_splitter.split_documents([corpus])
print(len(chunks))

Map: 청크별 요약을 생성합니다.

In [ ]:
# Map 과정 : 각 문서에 대해 요약을 생성합니다.
from tqdm import tqdm

map_prompt = ChatPromptTemplate([
    ('system', '''논문의 일부가 주어집니다.
해당 내용을 읽고 한국어로 요약하세요.
요약은 5개의 문단과 문단별 4~8개의 문장으로 작성하세요.
'''),
    ('user', '''{text}''')])

raw_summaries = []

map_chain  = map_prompt | llm | StrOutputParser()


for i in tqdm(range(len(chunks))):
    response = map_chain.invoke(chunks[i].page_content)

    raw_summaries.append(response)

    print('\n#',i)
    print(response)
    print('===========================')

In [ ]:
gathered_summaries = '\n---\n'.join(raw_summaries)

print(gathered_summaries)

In [ ]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
LLM 논문에 대한 요약문의 리스트가 주어집니다.
이를 읽고, 전체 주제를 포함하는 최종 요약을 작성하세요.
요약은 5개의 문단과 문단별 4-8개의 문장으로 작성하세요.
'''),
    ('user', '''{text}
---
Summary:
''')])

reduce_chain = reduce_prompt | llm | StrOutputParser()

summary = reduce_chain.invoke(gathered_summaries)

print(summary)